In [ ]:
import numpy as np
import cv2
from skimage.util import random_noise
import matplotlib.pyplot as plt

def compute_noise_index(image):
    """
    Computes a noise index for an image based on the minimum absolute difference 
    from each pixel to its 3x3 neighbors.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY).astype(np.float32)
    h, w = gray.shape
    dmin_vals = []

    for i in range(1, h - 1):
        for j in range(1, w - 1):
            patch = gray[i-1:i+2, j-1:j+2].flatten()
            center_pixel = gray[i, j]
            neighbors = np.delete(patch, 4)  # remove center pixel
            diffs = np.abs(neighbors - center_pixel)
            dmin = np.min(diffs)
            dmin_vals.append(dmin)

    noise_index = np.mean(np.square(dmin_vals))

    # Normalize to a scale of 1 to 10
    noise_score = np.clip((noise_index / 50) * 10, 1, 10)
    return noise_score

# Load image and resize
original_img = cv2.imread("MP_4_noise_estimate.png")
original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
original_img = cv2.resize(original_img, (224, 224))

# Add Gaussian noise
noisy_img = random_noise(original_img, mode='gaussian', var=0.01)
noisy_img = (255 * noisy_img).astype(np.uint8)

# Calculate noise scores
original_score = compute_noise_index(original_img)
noisy_score = compute_noise_index(noisy_img)

# Display
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(original_img)
axs[0].set_title(f"Original Image\nNoise Score: {original_score:.2f}")
axs[0].axis('off')

axs[1].imshow(noisy_img)
axs[1].set_title(f"Noisy Image\nNoise Score: {noisy_score:.2f}")
axs[1].axis('off')

plt.tight_layout()
plt.show()
